### 1. Import library

In [5]:
#This project use python 3.12.2 enviroment
#The enviroment helpful to analytics with libraries
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, TensorDataset
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


### 2. Read data

In [6]:
#Read the dates 
data = pd.read_csv("speed_data.csv")
data.head()
data = data.dropna()

In [7]:
#Separate target variable
data_y = data[data.columns[5]]
data_y.head()
#All other dates
data_x = data.drop(columns=["dec"])
data_x.head()

,gender,age,income,goal,career,attr,sinc,intel,fun,amb,shar,like,prob,met
0,0,21.0,69487.0,2.0,lawyer,6.0,9.0,7.0,7.0,6.0,5.0,7.0,6.0,2.0
1,0,21.0,69487.0,2.0,lawyer,7.0,8.0,7.0,8.0,5.0,6.0,7.0,5.0,1.0
3,0,21.0,69487.0,2.0,lawyer,7.0,6.0,8.0,7.0,6.0,8.0,7.0,6.0,2.0
4,0,21.0,69487.0,2.0,lawyer,5.0,6.0,7.0,7.0,6.0,6.0,6.0,6.0,2.0
5,0,21.0,69487.0,2.0,lawyer,4.0,9.0,7.0,4.0,6.0,4.0,6.0,5.0,2.0


##### 2.1 Normalize

In [8]:
#Standardization dates
data_x = pd.get_dummies(data_x)

#Scale values out of range
escalador = StandardScaler()
data_x = escalador.fit_transform(data_x)

##### 2.2 Shape 

In [9]:
#Shape data in train and test
data_x.shape[0]

X_train, X_test, Y_train,Y_test = train_test_split(data_x,data_y,test_size=0.2, random_state=2)

print("X Train: {}, X Test: {}, y_train {}, y_test {}".format(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape))

n_inputs = X_train.shape[1]

X Train: (2800, 217), X Test: (701, 217), y_train (2800,), y_test (701,)


##### 2.3 Transform to tensors

In [10]:
#transform the array of np or pnds to tensors
t_X_train = torch.from_numpy(X_train).float().to("cuda")
t_X_test = torch.from_numpy(X_test).float().to("cuda")
t_Y_train = torch.from_numpy(Y_train.values).float().to("cuda")
t_Y_test = torch.from_numpy(Y_test.values).float().to("cuda")
t_Y_train = t_Y_train[:,None]
t_Y_test = t_Y_test[:,None]

#define tensor test
test = TensorDataset(t_X_test,t_Y_test)

### 3 Create a CNN

In [11]:
class Red(nn.Module):
    def __init__(self, n_inputs):
        super(Red, self).__init__()
        self.linear1 = nn.Linear(n_inputs, 128)
        self.dropout1 = nn.Dropout(0.3)  
        self.linear2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.1)  
        self.linear3 = nn.Linear(64, 32)
        self.linear4 = nn.Linear(32, 1)

    def forward(self, inputs):
        x = F.relu(self.linear1(inputs))
        x = self.dropout1(x) 
        x = F.relu(self.linear2(x))
        x = self.dropout2(x)  
        x = F.relu(self.linear3(x))
        x = torch.sigmoid(self.linear4(x))
        return x

##### 3.1 Create a train function and test function

In [12]:
#Train function
def train(model, optimizer, loss_fn, t_X_train, t_Y_train, epochs, status_print):
    model.train()
    for epoch in range(1, epochs + 1):
        optimizer.zero_grad()
        Y_pred = model(t_X_train)
        loss = loss_fn(input=Y_pred, target=t_Y_train)
        loss.backward()
        optimizer.step()

        if epoch % status_print == 0:
            print(f"\nEpoch {epoch} \t Loss: {round(loss.item(), 4)}")

# Test function
def test(model, t_X_test, t_Y_test, status_print):
    model.eval()
    with torch.no_grad():
        Y_pred = model(t_X_test)
        Y_pred_class = Y_pred.round()
        correct = (Y_pred_class == t_Y_test).sum()
        accuracy = 100 * correct / float(len(t_Y_test))

        if status_print == 0:
            print("Accuracy: {}".format(accuracy.item()))
        return accuracy.item()

##### 3.3 Parameters of function

In [13]:
lr = 0.001
epochs = 2000
status_print = 100

#3.4 Inicialice model 

In [14]:
model = Red(n_inputs=n_inputs)
model.to("cuda")
loss_fn = nn.BCELoss()
optimizer = optim.Adam(params=model.parameters(), lr=lr)

### 4 Make a prediction with model

In [15]:
# Train and test model
print("Entrenando el modelo")
train(model, optimizer, loss_fn, t_X_train, t_Y_train, epochs, status_print)
print("Evaluando el modelo")
test(model, t_X_test, t_Y_test, status_print)

Entrenando el modelo

Epoch 100 	 Loss: 0.2349

Epoch 200 	 Loss: 0.1333

Epoch 300 	 Loss: 0.0655

Epoch 400 	 Loss: 0.032

Epoch 500 	 Loss: 0.0208

Epoch 600 	 Loss: 0.0209

Epoch 700 	 Loss: 0.0115

Epoch 800 	 Loss: 0.0068

Epoch 900 	 Loss: 0.0061

Epoch 1000 	 Loss: 0.0117

Epoch 1100 	 Loss: 0.0064

Epoch 1200 	 Loss: 0.0038

Epoch 1300 	 Loss: 0.0069

Epoch 1400 	 Loss: 0.0033

Epoch 1500 	 Loss: 0.0034

Epoch 1600 	 Loss: 0.0015

Epoch 1700 	 Loss: 0.0035

Epoch 1800 	 Loss: 0.0045

Epoch 1900 	 Loss: 0.0044

Epoch 2000 	 Loss: 0.0027
Evaluando el modelo


82.73894500732422